# Tensor network quantum control for expectation value minimisation

In this program, we take the optimised gate sequence from _vqe_from_expval_mpo.m_ and implement a state vector simulation to confirm they correctly minimising the expectation value.

For some reason, this is not working... even though we are correctly recovering the expectation value for the $n=4$ case. However, when we change the number of qubits, the expectation value starts failing too! wtf!

In [4]:
import numpy as np
import random
import itertools
import time
from matplotlib import pyplot as plt
from qutip import *

from pauli_string_functions_module_python import * 

In [11]:
#heisenberg model -- should be highly entangled ground state.
#when ZZ gates turned off, achieves -3.0000 minima.
#When ZZ gates then turned ON, achieves the true GS, -6.46
n = 4
XX_terms = local_2body_strings(n, 'XX') 
YY_terms = local_2body_strings(n, 'YY')
ZZ_terms = local_2body_strings(n, 'ZZ')

H_heis = 0
for i in range(n - 1):
    H_heis += operator_string_from_Pauli_string(XX_terms[i]) + \
              operator_string_from_Pauli_string(YY_terms[i]) + \
              operator_string_from_Pauli_string(ZZ_terms[i])
              
heis_min, heis_GS = H_heis.groundstate()

print(f'Ground state energy: {heis_min}')

GS_coeff = np.zeros(2**n, dtype = complex)
for i in range(2**n):
    GS_coeff[i] = heis_GS[i][0][0]

H_heis.data

Ground state energy: -6.464101615137754


<16x16 sparse matrix of type '<class 'numpy.complex128'>'
	with 40 stored elements in Compressed Sparse Row format>

In [13]:
import numpy as np
import qutip as qt
from scipy.sparse.linalg import eigsh

def heisenberg_hamiltonian(n, Jx=1.0, Jy=1.0, Jz=1.0):
    """
    Build the Heisenberg Hamiltonian:
    H = sum_j [ Jx X_j X_{j+1} + Jy Y_j Y_{j+1} + Jz Z_j Z_{j+1} ]
    for a chain of n spin-1/2 particles, open boundary conditions.
    """
    sx = qt.sigmax()
    sy = qt.sigmay()
    sz = qt.sigmaz()
    I  = qt.qeye(2)

    H = 0
    for j in range(n - 1):
        ops_X = [I] * n
        ops_Y = [I] * n
        ops_Z = [I] * n

        ops_X[j]   = sx
        ops_X[j+1] = sx
        ops_Y[j]   = sy
        ops_Y[j+1] = sy
        ops_Z[j]   = sz
        ops_Z[j+1] = sz

        H += Jx * qt.tensor(ops_X) + Jy * qt.tensor(ops_Y) + Jz * qt.tensor(ops_Z)

    return H  # stays sparse

def ground_state_energy(H):
    """
    Find the ground state energy using sparse iterative diagonalisation.
    """
    # Convert to CSR sparse matrix for scipy
    H_sparse = H.data  # qutip uses scipy.sparse underneath

    # Find smallest algebraic eigenvalue (ground state energy)
    E0, psi0 = eigsh(H_sparse, k=1, which='SA')  # SA = smallest algebraic
    return E0[0], qt.Qobj(psi0[:, 0])  # return energy and state as Qobj

# Example: n = 12 qubits
n_qubits = 10
H = heisenberg_hamiltonian(n_qubits)
E0, psi0 = ground_state_energy(H)

print(f"Ground state energy for n={n_qubits}: {E0:.6f}")


Ground state energy for n=10: -17.032141


In [138]:
#heisenberg model with single qubit control and native ZZ gates
n = 4
bin_num = 10
ctrl_num = 8

#pulse sequence data and manipulation into time bins
x_optm = np.loadtxt('heis_n=4_bin=1.csv', delimiter = ',')
T_optm = x_optm[-1]
Dt = T_optm / bin_num
cX_bin, cY_bin = np.zeros([n, bin_num]), np.zeros([n, bin_num])
for i in range(n):
    for j in range(bin_num):
        cX_bin[i, j] = x_optm[8 * j + 2 * i]
        cY_bin[i, j] = x_optm[8 * j + 2 * i + 1]
    cX_bin[i] = -np.flip(cX_bin[i])
    cY_bin[i] = -np.flip(cY_bin[i])

#time in simulation
time_steps = 20
times = np.linspace(0, T_optm, time_steps)

#pulse sequence for the simulation times
cX, cY = np.zeros([n, time_steps]), np.zeros([n, time_steps])
for i in range(n):
    count = 1
    for j in range(time_steps):
        if times[j] > count * Dt and count < bin_num:#second condition to avoid floating point errors
            count += 1
        cX[i, j] = cX_bin[i, count - 1]
        cY[i, j] = cY_bin[i, count - 1]

#control Hamiltonian for QuTIP mesovle function 
X_ops, Y_ops = np.zeros(n, dtype = list), np.zeros(n, dtype = list)
H_control = []
for i in range(n):
    X_ops[i] = operator_string_from_Pauli_string(pauli_string_at_qubit_position(n, 'X', i))
    Y_ops[i] = operator_string_from_Pauli_string(pauli_string_at_qubit_position(n, 'Y', i))
    H_control.append([X_ops[i], cX[i]])
    H_control.append([Y_ops[i], cY[i]])

#native Hamiltonian for QuTIP mesolve function
cZZ = np.ones(time_steps)
ZZ_ops = np.zeros(n, dtype = list)
H_native = []
for i in range(n - 1):
    ZZ_ops[i] = operator_string_from_Pauli_string(pauli_string_at_qubit_position(n, 'ZZ', i))
    H_native.append([ZZ_ops[i], cZZ])

H_time_dependent = H_control + H_native



#QuTIP simulation
psi0 = tensor([basis(2, 1)] * n)

result = sesolve(H_time_dependent, psi0, times)
psiT = result.states[-1]
psiT_energy = np.real((psiT.dag() * H_heis * psiT)[0][0][0])
print(f'GS energy: {heis_min}')
print(f'VQE state energy: {psiT_energy}')

#POTENTIALLY THE WRONG ORDER FOR OPTIMISATION COEFFICIENTS?!?!?!?!?!?!?!?!?!?!?!?!?!?!?



GS energy: -6.464101615137754
VQE state energy: -1.038243411238075


In [126]:
times

array([6.28505644e+00, 2.10238423e+02, 4.14191789e+02, 6.18145155e+02,
       8.22098521e+02, 1.02605189e+03, 1.23000525e+03, 1.43395862e+03,
       1.63791199e+03, 1.84186535e+03, 2.04581872e+03, 2.24977208e+03,
       2.45372545e+03, 2.65767882e+03, 2.86163218e+03, 3.06558555e+03,
       3.26953892e+03, 3.47349228e+03, 3.67744565e+03, 3.88139901e+03,
       4.08535238e+03, 4.28930575e+03, 4.49325911e+03, 4.69721248e+03,
       4.90116585e+03, 5.10511921e+03, 5.30907258e+03, 5.51302594e+03,
       5.71697931e+03, 5.92093268e+03, 6.12488604e+03, 6.32883941e+03,
       6.53279277e+03, 6.73674614e+03, 6.94069951e+03, 7.14465287e+03,
       7.34860624e+03, 7.55255961e+03, 7.75651297e+03, 7.96046634e+03,
       8.16441970e+03, 8.36837307e+03, 8.57232644e+03, 8.77627980e+03,
       8.98023317e+03, 9.18418654e+03, 9.38813990e+03, 9.59209327e+03,
       9.79604663e+03, 1.00000000e+04])

In [87]:
x = [1, 2, 3, 4, 5, 6]
x[:-1]

[1, 2, 3, 4, 5]

In [20]:
#crushed Ising model (cIm) - definition, spectrum and eigenstates
n = 4
X1, Xn = 'X' + 'I' * (n - 1), 'I' * (n - 1) + 'X'
cIm_terms = tuple(list(all_single_operator_strings(n, 'Z')) + [X1, Xn] + list(local_2body_strings(n, 'XX')))
H_cIm = 0 
for j in range(len(cIm_terms)):
    H_cIm += operator_string_from_Pauli_string(cIm_terms[j])

cIm_spectrum, cIm_states = H_cIm.eigenstates()
cIm_min, cIm_GS = cIm_spectrum[0], cIm_states[0]
print(f'Ground state energy: {cIm_min}')

Ground state energy: -5.457414830239131


In [21]:
#shortest vector problem (svp) style hamiltonian - definition, spectrum, eigenstates
#make sure order of weights and terms is the same as in the MATLAB code
n = 4
svp_terms = tuple(list(all_single_operator_strings(n, 'Z')) + list(nonlocal_2body_strings(n, 'Z', 'Z')))
svp_weights = ran(1, len(svp_terms))

H_svp = 0
for j in range(len(svp_terms)):
    H_svp += svp_weights[j] * operator_string_from_Pauli_string(svp_terms[j])

svp_spectrum, svp_states = H_svp.eigenstates()
svp_min, svp_GS = svp_spectrum[0], svp_states[0]

print(f'Ground state energy: {svp_min}')

Ground state energy: -2.3000823110299846


In [6]:
for i in range(2**n):
    if round(abs(svp_GS[i][0][0])) == 1:
        print(i)
#GS = 22 = 010110
gs = tensor([basis(2, 0), basis(2, 1), basis(2, 0), basis(2, 1)])#, basis(2, 1), basis(2, 0)])
fidelity(gs, svp_GS)

6


0.0

In [56]:
#importing and organising data
n = 4
bin_num = 10
ctrl_num = 8
x_optm = np.loadtxt('data/vqe_n=4_.csv', delimiter=',')
T_optm = x_optm[-1]
Dt = T_optm / bin_num
c_optm = np.reshape(x_optm[:-1], (bin_num, ctrl_num))

cX_temp, cY_temp = np.zeros([n, bin_num]), np.zeros([n, bin_num])
for i in range(n):
    cX_temp[i] = c_optm[:, 2 * i]
    cY_temp[i] = c_optm[:, 2 * i + 1]

#swapping sign and order of cX an cY
cX_binned, cY_binned = np.zeros([n, bin_num]), np.zeros([n, bin_num])
for i in range(n):
    cX_binned[i] = -np.flip(cX_temp[i])
    cY_binned[i] = -np.flip(cY_temp[i])
#row represents the time bin, collumn represents the qubit on which X or Y act

In [57]:
#state vector simulation parameters and time-dependent pulse definition
time_steps = 10000
times = np.linspace(0, T_optm, time_steps)

cX, cY = np.zeros([n, time_steps]), np.zeros([n, time_steps])
for i in range(n):
    count = 1
    for j in range(time_steps):
        if times[j] > count * Dt and count < bin_num: #second condition avoids floating error leading to count = bin_count + 1
            count += 1
        cX[i, j] = cX_binned[i, count - 1]
        cY[i, j] = cY_binned[i, count - 1]

X_ops, Y_ops = np.zeros(n, dtype = list), np.zeros(n, dtype = list)
H_control = []
for i in range(n):
    X_ops[i] = operator_string_from_Pauli_string(pauli_string_at_qubit_position(n, 'X', i))
    Y_ops[i] = operator_string_from_Pauli_string(pauli_string_at_qubit_position(n, 'Y', i))
    H_control.append([X_ops[i], cX[i]])
    H_control.append([Y_ops[i], cY[i]])

cZZ = np.zeros(time_steps)
ZZ_ops = np.zeros(n, dtype = list)
H_native = []
for i in range(n - 1):
    ZZ_ops[i] = operator_string_from_Pauli_string(pauli_string_at_qubit_position(n, 'ZZ', i))
    H_native.append([ZZ_ops[i], cZZ])

H_time_dependent = H_control + H_native

In [58]:
#state vector simulation --> does it work!?!
psi0 = tensor([basis(2, 0)] * n)
result = mesolve(H_time_dependent, psi0, times)
psiT = result.states[-1]

In [59]:
expect(H_svp, psiT), expect(H_svp, svp_GS)

(0.6126491884700125, -15.589104083569413)

In [60]:
fidelity(psiT, svp_GS)

0.1492692319057453

In [70]:
#testing crished Ising model for a single bin (ie; constant Hamiltonian)
#min energy: -5.104246e+00 > true GS, but close enough to test if we are realising what we want to realise 
n = 4

#crushed Ising model
X1, Xn = 'X' + 'I' * (n - 1), 'I' * (n - 1) + 'X'
cIm_terms = tuple(list(all_single_operator_strings(n, 'Z')) + [X1, Xn] + list(local_2body_strings(n, 'XX')))
H_cIm = 0 
for j in range(len(cIm_terms)):
    H_cIm += operator_string_from_Pauli_string(cIm_terms[j])
cIm_min = H_cIm.groundstate()[0]
print(f'Ground state energy: {cIm_min}')


Ground state energy: -5.45741483023913


In [77]:
#single qubit control with native ZZ gates (seemingly turned OFF!?!)
#with these settings; min energy = -5.104246e+00 != true ground state.
#reachability issue (?) due to no entangling operation

#optimal single pulse 
x_optm = np.loadtxt('cIm_1qcontrol_1bin_n=4.csv', delimiter = ',')
T_optm = x_optm[-1]
cX, cY = np.zeros(n), np.zeros(n)
for i in range(n):
    cX[i] = -x_optm[2 * i]
    cY[i] = -x_optm[2 * i + 1]
cZZ = 0

#optimal Hamiltonian
X_terms, Y_terms = all_single_operator_strings(n, 'X'), all_single_operator_strings(n, 'Y')
ZZ_terms = local_2body_strings(n, 'ZZ')
H_optm = 0
for i in range(n):
    H_optm += cX[i] * operator_string_from_Pauli_string(X_terms[i]) + cY[i] * operator_string_from_Pauli_string(Y_terms[i])
for i in range(n - 1):
    H_optm += cZZ * operator_string_from_Pauli_string(ZZ_terms[i])

#simulation of optimal pulse
psi0 = tensor([basis(2, 0)] * n)

steps = 10000
times = np.linspace(0, T_optm, steps)

result = mesolve(H_optm, psi0, times)
state = result.states[-1]
energy = state.dag() * H_cIm * state

print(f'Output state energy: {energy}')

#ISSUE: NO ZZ IS HAPPENING!!!!!! NO 2 QUBIT GATE IS HAPPENING ON THE MATLAB CODE!!!

Output state energy: Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[-5.1105407]]


In [75]:
#single and 2 qubit control -- is the two qubit control turned ON?!?!
#with these settings, min energy = -5.143259e+00 != true ground state
#initial seed guesses (x1 y1 x2 y2 ... z1z2 z2z3 z3z4  x'1 y'1 x'2 ... 'z'3z'4)

#c0 = np.loadtxt('data/initial_guesses.csv', delimiter = ',')


#optimal single and two qubit pulses
x_optm = np.loadtxt('cIm_2qcontrol_1bin_n=4.csv', delimiter = ',')
T_optm = x_optm[-1]
cX, cY = np.zeros(n), np.zeros(n)
for i in range(n):
    cX[i] = -x_optm[2 * i]
    cY[i] = -x_optm[2 * i + 1]
    
#cZZ = np.zeros(n - 1)
#for i in range(n - 1):
#    cZZ[i] = x_optm[2 * n + i]
cZZ = -np.ones(n - 1)

#optimal Hamiltonian
X_terms, Y_terms = all_single_operator_strings(n, 'X'), all_single_operator_strings(n, 'Y')
ZZ_terms = local_2body_strings(n, 'ZZ')
H_optm = 0
for i in range(n):
    H_optm += cX[i] * operator_string_from_Pauli_string(X_terms[i]) + cY[i] * operator_string_from_Pauli_string(Y_terms[i])

for i in range(n - 1):
    H_optm += cZZ[i] * operator_string_from_Pauli_string(ZZ_terms[i])

#simulation of optimal pulses
psi0 = tensor([basis(2, 0)] * n)

steps = 10000
times = np.linspace(0, T_optm, steps)

result = mesolve(H_optm, psi0, times)
state = result.states[-1]
energy = state.dag() * H_cIm * state

print(f'Output state energy: {energy}')




Output state energy: Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[3.5683291]]


In [33]:
#testing shortest vector problem for a single bin (ie; constant Hamiltonian)
x_optm = np.loadtxt('data/svp_1bin_n=4.csv', delimiter=',')
T_optm = x_optm[-1]
cX1, cY1 = -x_optm[0], -x_optm[1]
cX4, cY4 = -x_optm[-3], -x_optm[-2]
cZZ = -1

id = qeye(2)
X1, Y1 = tensor([sigmax(), id, id, id]), tensor([sigmay(), id, id, id])
X4, Y4 = tensor([id, id, id, sigmax()]), tensor([id, id, id, sigmay()])
Z1Z2 = tensor([sigmaz(), sigmaz(), id, id])
Z2Z3 = tensor([id, sigmaz(), sigmaz(), id])
Z3Z4 = tensor([id, id, sigmaz(), sigmaz()])

H_optm = cX1 * X1 + cY1 * Y1 + cX4 * X4 + cY4 * Y4 + cZZ * (Z1Z2 + Z2Z3 + Z3Z4)

psi0 = tensor([basis(2, 0)] * 4)

steps = 10000
times = np.linspace(0, T_optm, steps)

result = mesolve(H_optm, psi0, times)
state = result.states[-1]
#print(result.states[-1])
fidelity(result.states[-1], tensor([basis(2, 1), basis(2, 0), basis(2, 0), basis(2, 1)]))

0.031552401658149896